In [ ]:
cd /content/drive/MyDrive/internship_codes/vae

/content/drive/MyDrive/internship_codes/vae


In [ ]:
ls

all_vae/  models  __pycache__/  resnet_vae.py  results/  train.py  vae_model.py


In [ ]:
import torch
import torch.utils.data
from os import mkdir
from torch import optim
from torch.nn import functional as F
import os
from torchvision.utils import save_image
from torchvision.datasets import CelebA
from torch.utils.data import DataLoader
import random
from string import ascii_lowercase
import builtins
from torchvision import transforms
import torchvision.datasets as dset
from vae_model import VanillaVAE

In [ ]:
from vae_model import VanillaVAE

In [ ]:
CELEB_PATH= "/content/drive/MyDrive/internship_codes/dataset"
IMAGE_SIZE = 64
LATENT_DIM = 128
image_dim = 3 * IMAGE_SIZE * IMAGE_SIZE

In [ ]:
EPOCHS = 60  # number of training epochs
BATCH_SIZE = 16  # for data loaders
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('EPOCHS', EPOCHS, 'BATCH_SIZE', BATCH_SIZE, 'device', device)

EPOCHS 60 BATCH_SIZE 16 device cuda


In [ ]:
dataset = dset.ImageFolder(root=CELEB_PATH,
                           transform=transforms.Compose([
                               transforms.Resize(IMAGE_SIZE),
                               transforms.CenterCrop(IMAGE_SIZE),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                           ]))

In [ ]:
train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=8)
test_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False,num_workers=8)
model = VanillaVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# for model and results
directory = f'va'

def ini():
    if not os.path.exists("results"):
        os.makedirs("results")

    if not os.path.exists("results/"+directory):
        os.makedirs("results/"+directory)

def loss_function(recon_x, x, mu, log_var):
    MSE =F.mse_loss(recon_x, x)
    KLD = -0.5 * torch.mean(1 + log_var - mu.pow(2) - log_var.exp())
    kld_weight = 0.00025
    loss = MSE + kld_weight * KLD
    return loss

In [ ]:
def train(epoch):
    # for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        # print("hihi")
        torch.cuda.empty_cache()
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, log_var = model(data)
        log_var = torch.clamp_(log_var, -10, 10)
        loss = loss_function(recon_batch, data, mu, log_var)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, log_var = model(data)
            test_loss += loss_function(recon_batch, data, mu, log_var).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                        recon_batch.view(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE)[:n]])
                save_image(comparison.cpu(),
                           f'results/{directory}/reconstruction_{str(epoch)}.png', nrow=n)

In [ ]:

pet=f'results/{directory}'

ini()
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    torch.save(model, f'{pet}/vae_model_{epoch}.pth')
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, LATENT_DIM).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 3, IMAGE_SIZE, IMAGE_SIZE),
                    f'{pet}/sample_{str(epoch)}.png')

Train Epoch: 1 [0/9710 (0%)]	Loss: 0.046655
Train Epoch: 1 [160/9710 (2%)]	Loss: 0.026688
Train Epoch: 1 [320/9710 (3%)]	Loss: 0.020080
Train Epoch: 1 [480/9710 (5%)]	Loss: 0.018715
Train Epoch: 1 [640/9710 (7%)]	Loss: 0.016331
Train Epoch: 1 [800/9710 (8%)]	Loss: 0.020170
Train Epoch: 1 [960/9710 (10%)]	Loss: 0.016166
Train Epoch: 1 [1120/9710 (12%)]	Loss: 0.018074
Train Epoch: 1 [1280/9710 (13%)]	Loss: 0.017004
Train Epoch: 1 [1440/9710 (15%)]	Loss: 0.019460
Train Epoch: 1 [1600/9710 (16%)]	Loss: 0.016653
Train Epoch: 1 [1760/9710 (18%)]	Loss: 0.019381
Train Epoch: 1 [1920/9710 (20%)]	Loss: 0.017791
Train Epoch: 1 [2080/9710 (21%)]	Loss: 0.015863
Train Epoch: 1 [2240/9710 (23%)]	Loss: 0.018538
Train Epoch: 1 [2400/9710 (25%)]	Loss: 0.015072
Train Epoch: 1 [2560/9710 (26%)]	Loss: 0.017407
Train Epoch: 1 [2720/9710 (28%)]	Loss: 0.016912
Train Epoch: 1 [2880/9710 (30%)]	Loss: 0.019888
Train Epoch: 1 [3040/9710 (31%)]	Loss: 0.017072
Train Epoch: 1 [3200/9710 (33%)]	Loss: 0.016031
Train E

In [ ]:
from google.colab import drive
drive.mount('/content/drive')